<a href="https://colab.research.google.com/github/RahulManavalan/Machine-Learning/blob/master/Multivariate_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
pip install tensorflow==1.0 

In [0]:
import pandas as pd 
import numpy as np 
from sklearn import linear_model , datasets

In [0]:
def readFile(filename): 
  data = pd.read_csv(filename,sep=",",usecols=[0,5],names=["Date","Price"],header=0)

  data["Date"] = pd.to_datetime(data["Date"],format="%Y-%m-%d")
  data = data.sort_values(["Date"],ascending=True) 

  returns = data[[key for key in dict(data.dtypes) if dict(data.dtypes)[key] in ["float64","int64"]]].pct_change()

  return np.array(returns["Price"])[1:]

In [0]:
def read_xom_oil_xom_nasdaq_data():
  nasdaqData = readFile("/content/^GSPC.csv")
  oilData = readFile("/content/USO.csv")
  xomData = readFile("/content/XOM.csv")

  return(nasdaqData,oilData,xomData)

In [0]:
nasdaqData , oilData , xomData = read_xom_oil_xom_nasdaq_data() 

combined = np.vstack((nasdaqData,oilData)).T


In [56]:
xomNasdaqOilModel =  linear_model.LinearRegression() 

xomNasdaqOilModel.fit(combined,xomData)
xomNasdaqOilModel.score(combined,xomData)

print(xomNasdaqOilModel.coef_)
print(xomNasdaqOilModel.intercept_)

[0.74831623 0.08227063]
-0.0021519883600707827


In [0]:
import tensorflow as tf 

In [0]:
nasdaq_W = tf.Variable(tf.zeros([1,1]),name="nasdaq_W")
oil_W = tf.Variable(tf.zeros([1,1]),name="oil_W")
b = tf.Variable(tf.zeros([1]),name="b")

nasdaq_x = tf.placeholder(tf.float32 , [None,1] , name="nasdaq_x")
oil_x = tf.placeholder(tf.float32,[None,1],name="oil_x")

nasdaq_Wx = tf.matmul(nasdaq_x,nasdaq_W)
oil_Wx = tf.matmul(oil_x,oil_W)

y = nasdaq_Wx + oil_Wx + b 
y_ = tf.placeholder(tf.float32,[None,1])

cost = tf.reduce_mean(tf.square(y_ - y))

train_ste_ftrl = tf.train.FtrlOptimizer(1).minimize(cost)

In [0]:
all_x_nasdaq = nasdaqData.reshape(-1,1) 
all_x_oil = oilData.reshape(-1,1) 
all_x_xom = xomData.reshape(-1,1) 

dataset_size = len(oilData)

In [0]:
def trainwithMultiplePointsperEpoch(steps,train_step,batch_size):
  init = tf.global_variables_initializer() 

  with tf.Session() as sess:
    sess.run(init) 
    
    for i in range(steps):
      if dataset_size == batch_size:
        batch_start_index = 0 
      elif dataset_size<batch_size:
        raise ValueError("batch size must be lesser than the dataset size")
      else :
        batch_start_index = (i*batch_size)%dataset_size
      batch_end_index = batch_start_index + batch_size 

      batch_x_nasdaq = all_x_nasdaq[batch_start_index:batch_end_index]
      batch_x_oil = all_x_oil[batch_start_index:batch_end_index]
      batch_ys = all_x_xom[batch_start_index:batch_end_index]

      feed = {nasdaq_x:batch_x_nasdaq , oil_x : batch_x_oil , y_:batch_ys}

      sess.run(train_ste_ftrl,feed_dict = feed) 

      if (i+1)/500 == 0 : 
        print("After %d iterations :"%i) 
        print("W1: %s" %sess.run(nasdaq_W))
        print("W2: %s" %sess.run(oil_W))
        print("b : %s" %sess.run(b))

        print("cost function : %f" %sess.run(cost,feed_dict = feed))

In [0]:
trainwithMultiplePointsperEpoch(5000,train_ste_ftrl,len(oilData))